In [89]:
import pandas as pd

In [90]:
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")

In [91]:
# 便宜的に、緯度、経度が欠損の箇所には0を入れる
train["緯度"] = train["緯度"].fillna(0)
train["経度"] = train["経度"].fillna(0)

test["緯度"] = test["緯度"].fillna(0)
test["経度"] = test["経度"].fillna(0)

In [92]:
import numpy as np

# pandas seriesをnumpy arrayに変換
ido_keido_vecs = train[["緯度", "経度"]].values
kokuhous = train["is_kokuhou"].values

In [93]:
ido_keido_vec_with_kokuhou = [
    (ido_keido_vec, kokuhou) for ido_keido_vec, kokuhou in zip(ido_keido_vecs, kokuhous)
]

In [96]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

# ランダムな2次元データセットを生成
np.random.seed(0)
data_points = ido_keido_vecs


def get_nearest_kokuhou(ido_keido_vec, kokuhous, k=10) -> float:
    first_idx = 0

    if ido_keido_vec[0] == 0.0:
        return None

    # NearestNeighborsインスタンスを作成し、データセットをフィット
    neighbors = NearestNeighbors(n_neighbors=k + 1)  # k + 1(自身)
    neighbors.fit(data_points)

    # 選択した点の最も近いk件の近傍を見つける
    _, indices = neighbors.kneighbors([ido_keido_vec])

    print(ido_keido_vec)
    print(data_points[indices])

    # 自身を除いた近傍の平均を返す
    if (
        data_points[indices][0][0][0] == ido_keido_vec[0]
        and data_points[indices][0][0][1] == ido_keido_vec[1]
    ):
        print("find")
        first_idx = 1
    return kokuhous[indices[:, first_idx:]].mean()


train_output = []
for ido, keido in zip(train["緯度"], train["経度"]):
    res = get_nearest_kokuhou(ido_keido_vec=np.array((ido, keido)), kokuhous=kokuhous)
    train_output.append(res)
    break

# test_output = []
# for ido, keido in zip(test["緯度"], test["経度"]):
#     res = get_nearest_kokuhou(ido_keido_vec=np.array((ido, keido)), kokuhous=kokuhous)
#     test_output.append(res)
#     break

[ 34.69811 135.5214 ]
[[[ 34.69811    135.5214    ]
  [ 34.69507613 135.52504288]
  [ 34.69507613 135.52504288]
  [ 34.69507613 135.52504288]
  [ 34.69507613 135.52504288]
  [ 34.69507613 135.52504288]
  [ 34.69507613 135.52504288]
  [ 34.69507613 135.52504288]
  [ 34.69507613 135.52504288]
  [ 34.69507613 135.52504288]
  [ 34.69507613 135.52504288]]]
find


In [88]:
import pickle

with open("../eda/neighbor_train_feat.pkl", "wb") as f:
    pickle.dump(train_output, f)

with open("../eda/neighbor_test_feat.pkl", "wb") as f:
    pickle.dump(test_output, f)